In [ ]:
import numpy as np
import torch
import scipy.sparse as sp

def recall_at_k(gt_mat, results, k=20):
    recall_sum = 0
    num_users = gt_mat.shape[0]
    valid_users = 0

    # Verificamos si es rala para usar el acceso rápido, si no, usamos numpy estándar
    is_sparse = sp.issparse(gt_mat)
    if is_sparse:
        gt_mat = gt_mat.tolil()

    for i in range(num_users):
        if is_sparse:
            relevant_items = gt_mat.rows[i]
        else:
            relevant_items = np.where(gt_mat[i, :] > 0)[0]
            
        if len(relevant_items) == 0:
            continue
            
        valid_users += 1
        top_predicted_items = results[i, :k]
        
        hits = len(set(relevant_items).intersection(set(top_predicted_items)))
        recall_sum += hits / len(relevant_items)
        
    return recall_sum / valid_users if valid_users > 0 else 0


def ndcg_at_k(gt_mat, results, k=20):
    ndcg_sum = 0
    num_users = gt_mat.shape[0]
    valid_users = 0

    is_sparse = sp.issparse(gt_mat)
    if is_sparse:
        gt_mat = gt_mat.tolil()

    for i in range(num_users):
        if is_sparse:
            relevant_items = set(gt_mat.rows[i])
        else:
            relevant_items = set(np.where(gt_mat[i, :] > 0)[0])

        if len(relevant_items) == 0:
            continue
            
        valid_users += 1
        top_predicted_items = results[i, :k]
        
        dcg = 0
        idcg = 0
        
        for j, item in enumerate(top_predicted_items):
            if item in relevant_items:
                dcg += 1 / np.log2(j + 2)
        
        for j in range(min(len(relevant_items), k)):
            idcg += 1 / np.log2(j + 2)
            
        if idcg > 0:
            ndcg_sum += dcg / idcg
            
    return ndcg_sum / valid_users if valid_users > 0 else 0


def csr2torch(csr_matrix):
    """
    Convierte una matriz CSR de Scipy a un SparseTensor de PyTorch.
    Usa torch.sparse_coo_tensor para evitar advertencias de depreciación.
    """
    coo = csr_matrix.tocoo()
    indices = torch.LongTensor(np.vstack((coo.row, coo.col)))
    values = torch.FloatTensor(coo.data)
    shape = torch.Size(coo.shape)
    return torch.sparse_coo_tensor(indices, values, shape)


def normalize_sparse_adjacency_matrix(adj_matrix, alpha):
    """
    Normalización simétrica asimétrica: D^-alpha * A * D^-(1-alpha)
    Optimizado para operaciones ralas en PyTorch.
    """
    # Suma de filas y columnas usando mm rala
    # Creamos un vector de unos en el dispositivo de la matriz
    ones_col = torch.ones((adj_matrix.shape[1], 1), device=adj_matrix.device)
    ones_row = torch.ones((adj_matrix.shape[0], 1), device=adj_matrix.device)
    
    rowsum = torch.sparse.mm(adj_matrix, ones_col).squeeze()
    colsum = torch.sparse.mm(adj_matrix.t(), ones_row).squeeze()

    # Calcular inversas con potencias
    d_inv_rows = torch.pow(rowsum, -alpha)
    d_inv_rows[torch.isinf(d_inv_rows)] = 0.0
    
    d_inv_cols = torch.pow(colsum, alpha - 1)
    d_inv_cols[torch.isinf(d_inv_cols)] = 0.0

    # Crear matrices diagonales ralas
    size_r = rowsum.size(0)
    idx_r = torch.arange(size_r, device=adj_matrix.device).repeat(2, 1)
    d_mat_rows = torch.sparse_coo_tensor(idx_r, d_inv_rows, (size_r, size_r))
    
    size_c = colsum.size(0)
    idx_c = torch.arange(size_c, device=adj_matrix.device).repeat(2, 1)
    d_mat_cols = torch.sparse_coo_tensor(idx_c, d_inv_cols, (size_c, size_c))

    # Normalización: D_r * A * D_c
    # Nota: .mm en tensores ralos solo funciona con el primer argumento ralo y el segundo denso, 
    # o usando torch.sparse.mm. Para ralo-ralo se usa torch.sparse.mm
    norm_adj = torch.sparse.mm(d_mat_rows, adj_matrix)
    norm_adj = torch.sparse.mm(norm_adj, d_mat_cols)

    return norm_adj


def min_max_normalize(tensor):
    min_val = tensor.min()
    max_val = tensor.max()
    if max_val - min_val == 0:
        return tensor
    return (tensor - min_val) / (max_val - min_val)

# --- Funciones auxiliares adicionales si se requieren ---

def top_k(S, k=1):
    """Retorna los valores y los índices de las top-k recomendaciones."""
    return torch.topk(S, k, dim=1)

In [ ]:
import random
import numpy as np
import torch
import scipy.sparse as sp
import os
import gc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def run_tuning():
    dataset = "latam"
    batch_size = 500  
    sample_size = 20000
    
    print(f"🚀 Iniciando Afinamiento (Modo GPU-TopK Optimizado)")
    
    # 1. Cargar Matrices
    sp_tr = sp.load_npz(f"dataset/{dataset}_train.npz")
    sp_val = sp.load_npz(f"dataset/{dataset}_val.npz")
    num_total_authors = sp_tr.shape[0]

    np.random.seed(42)
    sample_indices = np.random.choice(num_total_authors, sample_size, replace=False)
    sample_indices.sort() 

    # Cargar matriz rala base y mover a GPU
    R_tr_gpu = csr2torch(sp_tr).to(device).float()

    configs = [(a, f) for a in [0.4, 0.5, 0.6, 0.7] for f in [1, 2, 3]]
    best_recall = 0
    best_params = None

    print(f"\n{'Alpha':<7} | {'Filtro':<7} | {'Recall@20':<10} | {'NDCG@20':<10}")
    print("-" * 55)

    for alpha, filt in configs:
        # Normalización
        R_norm = normalize_sparse_adjacency_matrix(R_tr_gpu, alpha).float()
        R_norm_T = R_norm.t()

        total_recall = 0
        total_ndcg = 0
        
        with torch.no_grad():
            for i in range(0, sample_size, batch_size):
                start = i
                end = min(i + batch_size, sample_size)
                curr_idx = sample_indices[start:end]
                
                # 1. Batch denso de entrenamiento para el cálculo
                R_batch_dense = torch.from_numpy(sp_tr[curr_idx].toarray()).to(device).float()
                
                # 2. Operaciones TurboCF (Proyección Asociativa)
                p1 = torch.sparse.mm(R_norm_T, R_batch_dense.t()).t()
                p2 = torch.sparse.mm(R_norm, p1.t()).t()

                if filt == 1:
                    batch_res = p2
                elif filt == 2:
                    p2_p = torch.sparse.mm(R_norm_T, p2.t()).t()
                    p2_p = torch.sparse.mm(R_norm, p2_p.t()).t()
                    batch_res = 2 * p2 - p2_p
                elif filt == 3:
                    p2_f = torch.sparse.mm(R_norm_T, p2.t()).t()
                    p2_f = torch.sparse.mm(R_norm, p2_f.t()).t()
                    p3_f = torch.sparse.mm(R_norm_T, p2_f.t()).t()
                    p3_f = torch.sparse.mm(R_norm, p3_f.t()).t()
                    batch_res = p2 + 0.01 * (-p3_f + 10*p2_f - 29*p2)

                # 3. Masking: Penalizar ítems ya vistos en entrenamiento
                batch_res.sub_(R_batch_dense * 1e9) 
                
                # 4. EXTRACCIÓN TOP-K EN GPU (Esto arregla el 0.0000 y da velocidad)
                _, top_indices = torch.topk(batch_res, k=20, dim=1)
                preds_indices = top_indices.cpu().numpy()
                
                # 5. Ground Truth Ralo para evaluación rápida
                gt_batch_sparse = sp_val[curr_idx]
                
                # 6. Evaluación usando las funciones optimizadas
                total_recall += recall_at_k(gt_batch_sparse, preds_indices, k=20) * (end - start)
                total_ndcg += ndcg_at_k(gt_batch_sparse, preds_indices, k=20) * (end - start)
                
                # Limpieza de temporales del batch
                del R_batch_dense, batch_res, p1, p2
                if 'p2_p' in locals(): del p2_p
                if 'p2_f' in locals(): del p2_f
                if 'p3_f' in locals(): del p3_f
                
                if (i // batch_size) % 10 == 0:
                    torch.cuda.empty_cache()
                
        cur_recall = total_recall / sample_size
        cur_ndcg = total_ndcg / sample_size

        print(f"{alpha:<7.2f} | {filt:<7} | {cur_recall:<10.4f} | {cur_ndcg:<10.4f}")

        if cur_recall > best_recall:
            best_recall = cur_recall
            best_params = (alpha, filt)

        del R_norm, R_norm_T
        gc.collect()
        torch.cuda.empty_cache()

    print("-" * 55)
    print(f"🏆 Ganador: Alpha={best_params[0]}, Filtro={best_params[1]}")

if __name__ == "__main__":
    run_tuning()

In [ ]:
import torch
import numpy as np
import scipy.sparse as sp
import os
import gc

# Configurar dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def run_final_evaluation():
    # --- CONFIGURACIÓN ---
    dataset = "latam"
    batch_size = 500
    # Usa los mejores parámetros que obtuviste en el tuning
    best_alpha = 0.40  
    best_filter = 2
    top_k_save = 100 # Cuántas recomendaciones guardar para tu plataforma
    
    print(f"🚀 Iniciando Evaluación Final y Generación de Activos (Alpha={best_alpha}, Filtro={best_filter})")
    
    # 1. Carga y Unión de Datos (Train + Val para entrenar el modelo final)
    sp_tr = sp.load_npz(f"dataset/{dataset}_train.npz")
    sp_val = sp.load_npz(f"dataset/{dataset}_val.npz")
    sp_test = sp.load_npz(f"dataset/{dataset}_test.npz") # Conjunto para métrica final

    sp_final_train = sp_tr + sp_val 
    num_authors = sp_final_train.shape[0]

    # Mover matriz rala de entrenamiento a GPU
    R_final_tr_gpu = csr2torch(sp_final_train).to(device).float()
    
    # 2. Normalización de Grafo
    print("🧠 Generando Normalización de Grafo...")
    R_norm = normalize_sparse_adjacency_matrix(R_final_tr_gpu, best_alpha).float()
    R_norm_T = R_norm.t()

    # Preparar contenedores para exportar a la API
    final_indices = np.zeros((num_authors, top_k_save), dtype=np.int32)
    
    total_recall_test = 0
    total_ndcg_test = 0
    valid_users_test = 0

    print(f"🔍 Procesando {num_authors:,} autores...")

    with torch.no_grad():
        for i in range(0, num_authors, batch_size):
            start = i
            end = min(i + batch_size, num_authors)
            
            # Batch denso de la matriz de entrenamiento final
            R_batch_dense = torch.from_numpy(sp_final_train[start:end].toarray()).to(device).float()
            
            # --- OPERACIONES TURBOCF (ASOCIATIVAS) ---
            # En lugar de usar P, usamos R_norm y R_norm_T secuencialmente
            p1 = torch.sparse.mm(R_norm_T, R_batch_dense.t()).t()
            p2 = torch.sparse.mm(R_norm, p1.t()).t()

            if best_filter == 1:
                batch_res = p2
            elif best_filter == 2:
                p2_p = torch.sparse.mm(R_norm_T, p2.t()).t()
                p2_p = torch.sparse.mm(R_norm, p2_p.t()).t()
                batch_res = 2 * p2 - p2_p
            elif best_filter == 3:
                p2_f = torch.sparse.mm(R_norm_T, p2.t()).t()
                p2_f = torch.sparse.mm(R_norm, p2_f.t()).t()
                p3_f = torch.sparse.mm(R_norm_T, p2_f.t()).t()
                p3_f = torch.sparse.mm(R_norm, p3_f.t()).t()
                batch_res = p2 + 0.01 * (-p3_f + 10*p2_f - 29*p2)

            # Masking: No recomendar lo que ya existe en train+val
            batch_res.sub_(R_batch_dense * 1e9)
            
            # Extraer Top-K
            _, top_indices = torch.topk(batch_res, k=top_k_save, dim=1)
            preds_indices_np = top_indices.cpu().numpy()
            
            # Guardar para la API (los 100 mejores)
            final_indices[start:end] = preds_indices_np
            
            # Evaluación sobre el Test Set (Métrica Real de la Tesis)
            gt_test_batch = sp_test[start:end]
            # Solo evaluamos si el usuario tiene algo en el test set
            total_recall_test += recall_at_k(gt_test_batch, preds_indices_np, k=20) * (end - start)
            total_ndcg_test += ndcg_at_k(gt_test_batch, preds_indices_np, k=20) * (end - start)

            # Limpieza
            del R_batch_dense, batch_res, p1, p2
            if 'p2_p' in locals(): del p2_p
            if 'p2_f' in locals(): del p2_f
            if 'p3_f' in locals(): del p3_f
            
            if i % (batch_size * 20) == 0:
                print(f"Progreso: {i/num_authors*100:.1f}%")
                torch.cuda.empty_cache()

    # 3. Reporte Final de Tesis
    final_recall = total_recall_test / num_authors
    final_ndcg = total_ndcg_test / num_authors
    
    print("\n" + "="*30)
    print(f"📊 RESULTADOS FINALES SOBRE TEST SET")
    print(f"Recall@20: {final_recall:.4f}")
    print(f"NDCG@20:   {final_ndcg:.4f}")
    print("="*30)

    # 4. Guardar archivos para la API / Sistema
    output_path = "prod_assets"
    os.makedirs(output_path, exist_ok=True)
    np.save(f"{output_path}/turbocf_recs_idx.npy", final_indices)
    print(f"💾 Activos guardados en '{output_path}/'. ¡Listo para producción!")

if __name__ == "__main__":
    run_final_evaluation()